# Task 1: Data exploration

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pylab import rcParams
import statsmodels.api as sm
from fbprophet import Prophet
pd.plotting.register_matplotlib_converters()
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from pmdarima.arima import auto_arima

## Reading the data

**Task**: Read the csv file *RetailNZ.csv*.
Open the dataset by reading the csv file in a Pandas dataframe

In [ ]:
retail_df = pd.read_csv('RetailNZ.csv', sep = ',')
retail_df

## Processing data with Pandas

**Tasks**:
* Convert the *Date* column to date format. 
* Sort by date
* Set the date as index

In [ ]:
retail_df["Time"] = pd.to_datetime(retail_df["Time"], format = '%Y-%m')  # converting date
retail_df = retail_df.sort_values(by = 'Time')  # sorting
retail_df = retail_df.set_index('Time')  # indexing

**Task**: We're going to start by only looking at the Clothing data.

you can also try to use other time series
* 'Footwear'
* 'Chemist'

In [ ]:
data_key = 'Clothing'
#data_key = 'Chemist'
#data_key = 'Footwear'
df = retail_df[[data_key]]

### Data visualization

**Tasks**: plot the data using a line chart

What is the general trend noticed and what are the visible patterns?

In [ ]:
plt.rcParams['figure.figsize'] = [15, 7.5]
df.plot.line()

Let's zoom a bit, for the most recent years. Can you see some visible patterns?

In [ ]:
df[-36:].plot.line()  # we take the last 36 months -> 3 years

### Differencing

**Tasks**: we'll try to get rid of seasonality & trend by differencing

First, let's see if differencing with previous element works.

In [ ]:
df.diff().plot.line()

It removed the trend, but seasonality stayed. How about differencing with the element 12months ago?

In [ ]:
df.diff(periods=12).plot.line()

## Time series correlation

Let's check our observations using autocorrelation & partial autocorrelation

**Tasks**: plot the autocorrelation and partial autocorrelation

In [ ]:
plot_acf(df, lags=24)
plt.show()

In [ ]:
plot_pacf(df, lags=24)
plt.show()

## Time series decomposition

We will deconstruct the time series into several components. Each component represents a category of patterns. This step is called **time series decomposition**.

**Tasks**: Decompose the time serie and understand the underlying concept of:
* Trend
* Seasonality
* Residuals also called noise or randomness

_Pay attention to the scale of the graphs_

In [ ]:
# try to change the 'model' parameter to multiplicative to see the difference
decomposition = sm.tsa.seasonal_decompose(df, model='additive')
fig = decomposition.plot()
plt.show()

# Task 2: Forecasting

For the purpose of this exercise, we're going to keep years 1995-2007 for training, and try to predict years 2008,2009 and 2010. We will then verify our predictions against the ground truth.

In [ ]:
train = df[:-36]  # trainset is from first data point to the last data point minus 36 (months)
test = df[-36:]  # testset is from last data point minue 36 months to the end

train.index = pd.to_datetime(train.index)
test.index = pd.to_datetime(test.index)

ground_truth = test.copy()

def mape(true_series: pd.core.series.Series, pred_series: pd.core.series.Series):
    # Mean absolute percentage error
    y_true, y_hat = true_series.values, pred_series.values
    return 100. * np.mean(np.abs((y_true - y_hat) / y_true))


def mase(true_series: pd.core.series.Series, pred_series: pd.core.series.Series):
    # Mean absolute scaled error
    y_true, y_hat = true_series.values, pred_series.values
    errors = np.sum(np.abs(y_true - y_hat))
    t = y_true.size
    scale = t/(t-1) * np.sum(np.abs(np.diff(y_true, axis=0)))
    
    return errors / scale

def plot_result(train, ground_truth, predictions, title=None):
    plt.plot(train,lw=2, label='train')
    plt.plot(ground_truth,lw=2, label='true')
    plt.plot(predictions, label='prediction')
    plt.title((title or '') + ' MAPE=' + str(mape(ground_truth, predictions)))
    plt.legend()
    plt.show()

### Exponential smoothing

**Exponential smoothing** makes predictions based on past observations, but over time the weights are assigned exponentially decreasing values.

**Tasks**:
* Instantiate and exponential smoothing model
* train it (fit)
* plot the predictions and the groundtruth
* Try to put the seasonal_period to 11, why is it an important argument ?

In [ ]:
model = ExponentialSmoothing(train.values, seasonal_periods = 12, seasonal = 'add', trend = 'add', damped=False)
fit = model.fit()
exp_forecast = pd.Series(fit.forecast(36))
exp_forecast.index = pd.to_datetime(test.index)
plot_result(train, ground_truth, exp_forecast, title = 'Exponential Smoothing')

## Using ets() from R by rpy2 package

R language provide a lot of libraries including all the most famous forecasting models
rpy2 is a package that allows us to run methods from R packages throug Python.

It is useful to know this trick since R provide more built-in models than Python.

Let's try to train an Exponential Smoothing model using wrapped R

In [ ]:
from typing import List, Tuple, Any
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from rpy2.robjects.conversion import localconverter
from rpy2.robjects import r, pandas2ri
from rpy2.robjects.vectors import IntVector, FloatVector
forecast = importr("forecast")
rstats = importr('stats')

def ets(time_series: pd.core.series.Series, h: int, start: int = 1, frequency: int = 12) -> Tuple[np.ndarray, Any]:
    """
    Args:
        time_series (pd.core.series.Series): time series
        h (int): the horizon (how many prediction do we want)
        start (int): time of first observation
        frequency (int): the number of observations per unit time (1: annual, 4: quartly, 12: monthly, ...)
    Returns:
        numpy.ndarray: array with predictions
        Any: R object with predictions details
    """
    time_series_r = rstats.ts(FloatVector(time_series.values), start=start, frequency=frequency)
    result = forecast.forecast(forecast.ets(time_series_r, allow_multiplicative_trend=True), h=h)
    preds = np.asarray(result[1])
    return preds, result

In [ ]:
preds, ets_model = ets(train, h=36)
ets_forecast = pd.Series(preds)
ets_forecast.index = pd.to_datetime(test.index)
plot_result(train, ground_truth, ets_forecast, title = 'ETS')

In [ ]:
ets_model  # print some of the internals parameters of the trained model

### About seasonality: ARIMA and SARIMA

If we don't account for seasonality, we will only get predictions that take into account the general trend.

* Autoregressive (AR),
* Moving Average (MA),
* Autoregressive Moving Average (ARMA),
* Autoregressive Integrated Moving Average (ARIMA)
* Seasonal Autoregressive Integrated Moving Average (SARIMA)

ARIMA and ARIMA, both are Autoregressive models


In [ ]:
model = ARIMA(train.values, order=(5,1,0)) # check with (12,0,0)
fitarima = model.fit()
arima_forecast = pd.Series(fitarima.forecast(36)[0])
arima_forecast.index = pd.to_datetime(test.index)
plot_result(train, ground_truth, arima_forecast, title = 'ARIMA')

**SARIMA (Seasonal Autoregressive Integrated Moving Average)** - we will now take into account that there is a visible yearly pattern.

In [ ]:
model_SARIMAX = SARIMAX(train.values, order = (2,0,0), seasonal_order = (2,0,1,12), enforce_stationarity=False)
fitsarima = model_SARIMAX.fit()
sarimax_forecast = pd.Series(fitsarima.forecast(36))
sarimax_forecast.index = pd.to_datetime(test.index)
plot_result(train, ground_truth, sarimax_forecast, title = 'SARIMAX')

**Auto ARIMA** - we will now try to use automated version of best ARIMA fit.

In [ ]:
model_autoarima = auto_arima(train.values, seasonal=True, m=12, suppress_warnings=True)
pred = model_autoarima.predict(n_periods=36)
autoarima_forecast = pd.Series(pred)
autoarima_forecast.index = pd.to_datetime(test.index)
plot_result(train, ground_truth, autoarima_forecast, title = 'Auto ARIMA')

## Task 3: Evaluate models

We can use different metrics to evaluate the performances of our models, for instance

* MAPE (Mean Absolute Percentage Error)
* MASE (Mean Absolute Scaled Error)

**Tasks**: compute MAPE and MASE to compare SARIMA and Exponential Smoothing
Which one is the best?


In [ ]:
models = ["Exponential Smoothing", "AutoARIMA", "ETS", "SARIMAX"]
predictions = [exp_forecast, autoarima_forecast, ets_forecast, sarimax_forecast]
metrics = ["MAPE", "MASE"]
eval_funcs = [mape, mase]
for model_label, pred in zip(models, predictions):
    for metric, eval_func in zip(metrics, eval_funcs):
        print("Model: {}, Metric: {}, Value: {}".format(model_label, metric, eval_func(ground_truth, pred)))

# Optional: Playground

**Task**: try taking a look at the data for *Chemist* or *Footwear* and see how the seasonal patterns change and how adjusting the parameters can change your results.

Go back to the top of the notebook to change what time serie is used